# Generate rxns that only you need

Han, Kehang (hkh12@mit.edu)

## Set-up

In [ ]:
import os
from rmgpy.rmg.main import RMG, CoreEdgeReactionModel
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings

In [ ]:
# set-up RMG object
rmg = RMG()

In [ ]:
# load kinetic database and forbidden structures
rmg.database = RMGDatabase()
path = os.path.join(settings['database.directory'])

In [ ]:
# set up kinetics db
from rmgpy.data.rmg import database
# forbidden structure loading
database.loadForbiddenStructures(os.path.join(path, 'forbiddenStructures.py'))
# kinetics family Disproportionation loading
database.loadKinetics(os.path.join(path, 'kinetics'), kineticsFamilies='default', reactionLibraries=[],kineticsDepositories=[])

In [ ]:
# set up thermo and transport db
database.loadThermo(os.path.join(path, 'thermo'), thermoLibraries=['primaryThermoLibrary'])
database.loadTransport(os.path.join(path, 'transport'))

In [ ]:
# set up chemkin file saving condition
from rmgpy.chemkin import ChemkinWriter
from rmgpy.rmg.output import OutputHTMLWriter

# set rmg kinetics estimator
rmg.reactionModel = CoreEdgeReactionModel()
rmg.reactionModel.kineticsEstimator = "rate rules"

rmg.saveEdgeSpecies = True

## Input

In [ ]:
from rmg_wiki.utilities.reactionGenerator import loadInput, pathwayGenerator, loadSpeciesDictionary
pathway_inputfile = 'rxn_gen/surrogate/input.dat'
pathway_spec_dictfile = 'rxn_gen/surrogate/species_dictionary.txt'
output_path = os.path.dirname(pathway_inputfile)
rmg.outputDirectory = output_path
rmg.attach(ChemkinWriter(output_path))
rmg.attach(OutputHTMLWriter(output_path))

family_list, reactants_list, products_list = loadInput(pathway_inputfile)
speciesDict = loadSpeciesDictionary(pathway_spec_dictfile)

In [ ]:
# create input species and family of interest
from rmgpy.rmg.model import Species
for family, reactants_str, products_str in zip(family_list, reactants_list, products_list):
    reactants = []
    products = []
    for reactant_str in reactants_str:
        try:# try if reactant_str is a smiles
            reactants.append(Species().fromSMILES(reactant_str))
        except:
            reactants.append(speciesDict[reactant_str])
    for product_str in products_str:
        try:
            products.append(Species().fromSMILES(product_str))
        except:
            products.append(speciesDict[product_str])

    # gen thermo and transport for reactants
    for spec in reactants:
        spec.generateThermoData(database)
        spec.generateTransportData(database)
    # pathway generation
    pathwayGenerator(reactants, products, rmg, [family])

## Output

In [ ]:
# save chemkin file for this step
rmg.reactionModel.outputSpeciesList.extend(rmg.reactionModel.edge.species)
rmg.reactionModel.outputReactionList.extend(rmg.reactionModel.edge.reactions)
rmg.saveEverything()